In [5]:
%run bollywood_actors_dataflow.py

In [86]:
%run bollywood_dataflow.py

In [87]:
%run bollywood_actress_dataflow.py

In [64]:
%run name_basics_dataflow.py

In [2]:
%run genres_join_dataflow.py

In [39]:
%run genres_dataflow.py

In [67]:
!bq query --nouse_legacy_sql \
"create or replace table datamart.genre_join_dataflow as select g.genre_id, j.tconst from datamart.genre_dataflow g join datamart.genre_join_dataflow j on g.genre=j.genre"

Waiting on bqjob_r1efa40e98017ac44_0000017964077c52_1 ... (22s) Current status: DONE   
Replaced ilitzkyzhou.datamart.genre_join_dataflow



In [55]:
!bq query --nouse_legacy_sql \
"select firstName, lastName from datamart.bollywood_actors_dataflow group by firstName, lastName having count(*) > 1"

Waiting on bqjob_r15f8d2705bae6ea5_0000017963d3e8f4_1 ... (0s) Current status: DONE   


In [60]:
!bq query --nouse_legacy_sql \
"select firstName, lastName from datamart.bollywood_actress_dataflow group by firstName, lastName having count(*) > 1"

Waiting on bqjob_r403b0c0a03936f9b_0000017963e30101_1 ... (0s) Current status: DONE   


In [62]:
!bq query --nouse_legacy_sql \
"select Title, Director from datamart.bollywood_dataflow group by Title, Director having count(*) > 1"

Waiting on bqjob_r196267811792f5aa_0000017963e78325_1 ... (0s) Current status: DONE   


In [65]:
!bq query --nouse_legacy_sql \
"select nconst from datamart.name_basics_dataflow group by nconst having count(*) > 1"

Waiting on bqjob_r64987ef13634702c_000001796404a277_1 ... (9s) Current status: DONE   


In [68]:
!bq query --nouse_legacy_sql \
"select genre_id, tconst from datamart.genre_join_dataflow group by genre_id,tconst having count(*) > 1"

Waiting on bqjob_r5b66b60a55a30943_00000179640a3db7_1 ... (14s) Current status: DONE   


In [69]:
!bq query --nouse_legacy_sql \
"select genre_id from datamart.genre_dataflow group by genre_id having count(*) > 1"

Waiting on bqjob_r46f38b93b90e3f97_00000179640a8478_1 ... (0s) Current status: DONE   


In [71]:
!bq query --nouse_legacy_sql \
"select a.firstName, a.lastName from datamart.bollywood_actress_dataflow a left outer join datamart.bollywood_dataflow b on b.Title=a.Debut_aslead_role where b.Title is null"

Waiting on bqjob_r3a48ffc0a50b7a62_000001796413f77a_1 ... (0s) Current status: DONE   


Distinct primary keys and valid child table references in the tables hold.

Due to the limited structure of the bollywood dataset, it is relatively difficult to get any meaningful cross-dataset queries from the data.

The goal within this query is to determine whether there exists a meaningful correlation between the release date of a bollywood title and the average Rating and number of reviews. This is one by doing a simple join between the bollywood_dataflow table and the titles table.

In [7]:
!bq query --nouse_legacy_sql \
"select b.Numerical_Date, b.Release_Date, t.averageRating, t.numVotes from datamart.bollywood_dataflow b \
join datamart.title t on t.primaryTitle=b.Title where b.Numerical_Date is not null and t.averageRating is not null limit 10"

Waiting on bqjob_r246e366474b7b1ae_0000017967bc54d9_1 ... (0s) Current status: DONE   
+----------------+--------------+---------------+----------+
| Numerical_Date | Release_Date | averageRating | numVotes |
+----------------+--------------+---------------+----------+
|         736666 |   2018-04-06 |           5.7 |        7 |
|         735362 |   2014-09-12 |           5.1 |     2574 |
|         733276 |   2008-12-26 |           7.5 |    13475 |
|         733756 |   2010-04-16 |           3.1 |      485 |
|         734621 |   2012-08-31 |           7.1 |      369 |
|         734182 |   2011-06-17 |           4.0 |     1289 |
|         735498 |   2015-01-23 |           8.0 |    53165 |
|         736543 |   2017-12-08 |           6.3 |       35 |
|         735713 |   2015-08-28 |           6.1 |      453 |
|         735575 |   2015-04-10 |           3.7 |     1679 |
+----------------+--------------+---------------+----------+


In [8]:
!bq query --nouse_legacy_sql \
"create or replace view reports.rating_over_time as select b.Numerical_Date, b.Release_Date, t.averageRating, t.numVotes from datamart.bollywood_dataflow b \
join datamart.title t on t.primaryTitle=b.Title where b.Numerical_Date is not null and t.averageRating is not null"

Waiting on bqjob_r50b940c3d375d1b8_0000017967bd4fa0_1 ... (0s) Current status: DONE   
Replaced ilitzkyzhou.reports.rating_over_time



In [1]:
!bq query --nouse_legacy_sql \
"create view reports.rating_by_actress as select a.firstName, a.lastName, t.primaryTitle, t.averageRating, t.numVotes from datamart.bollywood_actress_dataflow a join \
(select primaryTitle, avg(averageRating) as averageRating, sum(numVotes) as numVotes from datamart.title group by primaryTitle) t on t.primaryTitle=a.Debut_aslead_role \
where t.averageRating is not null order by averageRating desc"

Waiting on bqjob_r2adae44598044ce6_00000179676162ef_1 ... (0s) Current status: DONE   
Created ilitzkyzhou.reports.rating_by_actress



We want to see if specific bollywood directors produce better received titles than others. This is done by joining the bollywood_dataflow table with the title table, and comparing the averageRating of each director by grouping by the Director. However, it is possible that the average rating is skewed for some directors with very low movies produced, so we filter out any director who has less than 5 movies directed.

In [97]:
!bq query --nouse_legacy_sql \
"select b.Director, avg(t.averageRating) as averageRating, count(*) as numMovies from datamart.bollywood_dataflow b join datamart.title t on t.primaryTitle=b.Title \
group by b.Director having numMovies >= 5 order by averageRating desc limit 10"

Waiting on bqjob_r36eafab221f8d2c3_0000017964b3e37c_1 ... (1s) Current status: DONE   
+------------------------+-------------------+-----------+
|        Director        |   averageRating   | numMovies |
+------------------------+-------------------+-----------+
| I. P. Tiwari           |               9.0 |         8 |
| Ezra Mir               |               8.7 |        19 |
| M. Sadiq, S.U. Sunny   |              8.52 |        24 |
| A. N. Bannerjee        |               8.3 |         6 |
| H. M. Reddy            |               8.2 |         5 |
| Sampooran Singh Gulzar |               8.2 |        11 |
| Farhan Akhtar          | 8.133333333333333 |         7 |
| V. M. Gunjal           |               8.1 |        12 |
| Rajkumar Hirani        |              8.04 |         8 |
| Mamasaheb Shinde       |               8.0 |         6 |
+------------------------+-------------------+-----------+


In [2]:
!bq query --nouse_legacy_sql \
"create view reports.rating_by_director as select b.Director, avg(t.averageRating) as averageRating, count(*) as numMovies from datamart.bollywood_dataflow b join datamart.title t on t.primaryTitle=b.Title \
group by b.Director having numMovies >= 5 order by averageRating desc limit 10"

Waiting on bqjob_r22b08305e28edb01_000001796761c19f_1 ... (0s) Current status: DONE   
Created ilitzkyzhou.reports.rating_by_director



We are aiming to see whether the month a movie was released in has any impact on its average rating, or the number of votes. This is done by joining the bollywood_dataflow table with the title table, and grouping the average rating and the total number of votes with the release month, which tells us the average rating and the total number of votes per month.

In [51]:
!bq query --nouse_legacy_sql \
"select b.Release_Month, avg(t.averageRating) as averageRating, sum(t.numVotes) as numVotes from datamart.bollywood_dataflow b \
join datamart.title t on t.primaryTitle=b.Title group by b.Release_Month having b.Release_Month is not null order by Release_Month"

Waiting on bqjob_r6ea0e85ed952cc58_0000017963bac592_1 ... (1s) Current status: DONE   
+---------------+--------------------+----------+
| Release_Month |   averageRating    | numVotes |
+---------------+--------------------+----------+
|             1 |  6.611343283582088 |  1018162 |
|             2 | 6.4373493975903635 |  1110583 |
|             3 |   6.70439276485788 |   596763 |
|             4 |  6.665902578796562 |   459825 |
|             5 | 6.5424124513618676 |   855088 |
|             6 |  6.353723404255319 |   556955 |
|             7 |  6.320652173913045 |   786715 |
|             8 | 6.5854838709677415 |  1907714 |
|             9 |  6.599770642201834 |  1437652 |
|            10 |  6.776237623762376 |  1940828 |
|            11 | 6.2575581395348845 |   603364 |
|            12 |  6.628671328671329 |  1384102 |
+---------------+--------------------+----------+


In [3]:
!bq query --nouse_legacy_sql \
"create view reports.rating_by_month as select b.Release_Month, avg(t.averageRating) as averageRating, sum(t.numVotes) as numVotes from datamart.bollywood_dataflow b \
join datamart.title t on t.primaryTitle=b.Title group by b.Release_Month having b.Release_Month is not null order by Release_Month"

Waiting on bqjob_r70883f118517fe31_0000017967623b59_1 ... (0s) Current status: DONE   
Created ilitzkyzhou.reports.rating_by_month



This isn't a cross-dataset query, I'm just doing it out of personal interest. Checks average Rating and popularity by genre.

In [16]:
!bq query --nouse_legacy_sql \
"select g.genre, avg(t.averageRating) as averageRating, sum(t.numVotes) as numVotes from datamart.genre_dataflow g join datamart.genre_join_dataflow j on g.genre_id=j.genre_id \
join datamart.title t on t.tconst=j.tconst group by g.genre order by averageRating desc"

Waiting on bqjob_r3821c1aa89dc6ce1_0000017967cb4080_1 ... (0s) Current status: DONE   
+-------------+--------------------+-----------+
|    genre    |   averageRating    | numVotes  |
+-------------+--------------------+-----------+
| History     |  7.332446937432914 |  33303203 |
| Documentary | 7.2289979594811244 |  18929717 |
| Mystery     |  7.181210583132897 | 112134515 |
| Crime       |  7.158099932196317 | 213624702 |
| Biography   |  7.150906492810576 |  62171212 |
| Western     |  7.094348390623934 |   9558419 |
| Fantasy     |   7.07809810112894 | 103890433 |
| Adventure   |  7.064965768439516 | 279336191 |
| Animation   |  7.045950224664169 |  80101786 |
| Drama       | 7.0332248613497494 | 558512378 |
| Family      |  7.031189079208857 |  55074286 |
| War         |  7.006997037867262 |  19999179 |
| Action      |  6.977362172830778 | 335529337 |
| Sport       |  6.964842534992224 |  17190520 |
| Comedy      |  6.935881993736659 | 331498753 |
| Music       |  6.924303936863

In [12]:
!bq query --nouse_legacy_sql \
"create view reports.rating_by_genre as select g.genre, avg(t.averageRating) as averageRating, sum(t.numVotes) as numVotes from datamart.genre_dataflow g join datamart.genre_join_dataflow j on g.genre_id=j.genre_id \
join datamart.title t on t.tconst=j.tconst group by g.genre order by averageRating desc"

Waiting on bqjob_r7b8eea519b465e6d_0000017967c3dc47_1 ... (0s) Current status: DONE   
Created ilitzkyzhou.reports.rating_by_genre

